In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip


In [12]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [13]:
# Load dataset
train_file_path = '/kaggle/input/quora-question-pairs/train.csv.zip'
test_file_path = '/kaggle/input/quora-question-pairs/test.csv'

# Load CSV files
train_df = pd.read_csv(train_file_path, compression='zip')
test_df = pd.read_csv(test_file_path)

In [14]:
df_train, df_val = train_test_split(train_df, test_size=0.9, random_state=42)

In [15]:
class MNRankingLoss(torch.nn.Module):
    def __init__(self, model_name):
        super(MNRankingLoss, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        # Get CLS token embeddings
        outputs1 = self.model(input_ids1, attention_mask=attention_mask1)
        outputs2 = self.model(input_ids2, attention_mask=attention_mask2)
        embeddings1 = outputs1.last_hidden_state[:, 0, :]
        embeddings2 = outputs2.last_hidden_state[:, 0, :]
        return embeddings1, embeddings2

In [16]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    question1_list = df['question1'].astype(str).tolist()
    question2_list = df['question2'].astype(str).tolist()
    
    # Tokenize the inputs
    encodings = tokenizer(
        text=question1_list,
        text_pair=question2_list,
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    
    input_ids1 = encodings['input_ids']
    attention_mask1 = encodings['attention_mask']
    input_ids2 = encodings['token_type_ids']  # Use token_type_ids as input_ids2
    attention_mask2 = encodings['attention_mask']
    
    labels = torch.tensor(df['is_duplicate'].values)
    
    dataset = TensorDataset(input_ids1, attention_mask1, input_ids2, attention_mask2, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [18]:
model_name = 'bert-base-uncased'
bi_encoder_mnrl = MNRankingLoss(model_name)
tokenizer = bi_encoder_mnrl.tokenizer
max_len = 128
batch_size = 16

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please u

In [19]:
train_loader = create_data_loader(df_train, tokenizer, max_len, batch_size)
val_loader = create_data_loader(df_val, tokenizer, max_len, batch_size)

In [20]:
optimizer = torch.optim.Adam(bi_encoder_mnrl.parameters(), lr=1e-5)

In [21]:
def multiple_negative_ranking_loss(embeddings1, embeddings2):
    # Compute cosine similarity
    similarities = F.cosine_similarity(embeddings1.unsqueeze(1), embeddings2.unsqueeze(0), dim=-1)
    
    # Apply softmax to get log probabilities
    logits = similarities / 0.1  # Temperature scaling
    labels = torch.arange(len(logits)).to(logits.device)
    loss = F.cross_entropy(logits, labels)
    return loss

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bi_encoder_mnrl.to(device)

MNRankingLoss(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [23]:
for epoch in range(1):  # Example: 3 epochs
    bi_encoder_mnrl.train()
    running_loss = 0.0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
        input_ids1, attention_mask1, input_ids2, attention_mask2, _ = batch
        input_ids1, attention_mask1 = input_ids1.to(device), attention_mask1.to(device)
        input_ids2, attention_mask2 = input_ids2.to(device), attention_mask2.to(device)

        optimizer.zero_grad()
        embeddings1, embeddings2 = bi_encoder_mnrl(input_ids1, attention_mask1, input_ids2, attention_mask2)
        loss = multiple_negative_ranking_loss(embeddings1, embeddings2)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss / len(train_loader)}")

Epoch 1: 100%|██████████| 2527/2527 [27:44<00:00,  1.52it/s]

Epoch 1 Loss: 0.9790044151184631


In [24]:
    bi_encoder_mnrl.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validation'):
            input_ids1, attention_mask1, input_ids2, attention_mask2, _ = batch
            input_ids1, attention_mask1 = input_ids1.to(device), attention_mask1.to(device)
            input_ids2, attention_mask2 = input_ids2.to(device), attention_mask2.to(device)

            embeddings1, embeddings2 = bi_encoder_mnrl(input_ids1, attention_mask1, input_ids2, attention_mask2)
            loss = multiple_negative_ranking_loss(embeddings1, embeddings2)
            val_loss += loss.item()
    
    print(f"Validation Loss: {val_loss / len(val_loader)}")

Validation: 100%|██████████| 22742/22742 [1:30:39<00:00,  4.18it/s]

Validation Loss: 0.5859609146725061
